### 前言

🗯 国内疫情基本过去了，大部分地区已经恢复了该吃吃，该喝喝的状态；

💢 不过大洋彼岸的美国疫情似乎还是没有得到有效控制，每日新增病例还是5位数；

👋 本次项目就来汇总一下美国的疫情数据，数据来源[美国各州各城市2019新型冠状病毒🦠COVID19数据](https://www.kesci.com/home/dataset/5e844c33246a590036b97646)；

✅ **代码基于Pyecharts V1.7.1，代码均已运行通过~**

---

📩  **欢迎订阅我的专栏[【🏝 数据之美】](https://www.kesci.com/home/column/5e5cef70704543002c985797)**

📩  **欢迎订阅我的新专栏[【🏝 决战潘大师】](https://www.kesci.com/home/column/5f49f25214f35900368b11f8)**,尽量会保持周更；

----

❗️ 为不影响阅读体验，我将代码隐藏了～

🏝 **需要代码的同学Fork之后在自己的Klab中【运行】，然后右上角选择【查看】-【隐藏/显示所有代码输入】就可以看到代码了**～

---

相关教程：
* ✨[【pyecharts】50个完整例子,带你玩转可视化～](https://www.kesci.com/mw/project/5faf844d7d1e6d0030d75665)

* ✨[**【pyecharts教程】应该是全网最全的教程了～**](https://www.kesci.com/home/project/5eb7958f366f4d002d783d4a)

* ✨[【Pyecharts Gallery】中看不中用的可视化作品集合～](https://www.kesci.com/home/project/5e4bd73f80da780037be6b61)


----

**🔴 广告位：**
* 🔥[LOL数据分析：英雄联盟2020春/夏季赛数据可视化～](https://www.kesci.com/mw/project/5f93926ee0eb3e003be0304c)

* 🔥[**【pyecharts】美国疫情数据&特朗普关于新冠的发言～**](https://www.kesci.com/mw/project/5fbbbcc6d3251d00303bba30)

* 🔥[ **【Pyecharts】🏆 湖人2019-20赛季投篮数据可视化~**](https://www.kesci.com/home/project/5f24d178d278b1002c23b323)

* 🔥[ 2021年最新世界大学排名，来看看你的母校上榜没～](https://www.kesci.com/home/project/5ef4246163975d002c8f3bbb)



In [1]:
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.datasets import register_url
import pandas as pd

In [2]:
from pyecharts.globals import CurrentConfig

CurrentConfig.ONLINE_HOST = "https://cdn.kesci.com/lib/pyecharts_assets/"

In [3]:
# 美国总数据
df_total = pd.read_csv('/home/kesci/input/uscovid198505/us.csv')
# 州级别
df_state = pd.read_csv('/home/kesci/input/uscovid198505/us-states.csv')
# 城市级别
df_city = pd.read_csv('/home/kesci/input/uscovid198505/us-counties.csv')

### 疫情概览

#### 确诊

In [4]:
# 加载美国地图
register_url("https://echarts-maps.github.io/echarts-countries-js/")

state_list = ['California', 'Texas', 'New York', 'Florida', 'Arizona', 'Illinois']
loc = {
    'California': [-125.24302, 40.059232],
    'New York': [-77.875175, 42.850458],
    'Florida': [-82.269594, 27.785481],
    'Texas': [-101.102854, 31.916611],
    'Arizona': [-113.24302, 35.059232],
    'Illinois': [-90.5213, 39.8425]
}


data_new = df_state[df_state.date == df_state.date.max()]

data_pair = []
for _, row in data_new.iterrows():
    data_pair.append([row.state, row.cases])

# 用于geo和bar中的标签显示
fmt_js = """function (params) {return params.name+':'+Number(params.value[2]);}"""

"""
Bar:
展示每天top10国家的数据
放在grid的左下角
"""
t = sorted(data_pair, key=lambda x: x[1], reverse=True)[:10][::-1]

bar = Bar()
bar.add_xaxis([x for x, y in t])
bar.add_yaxis("", [y for x, y in t],
              itemstyle_opts={'normal': {'borderColor': '#1773c3',
                                         'shadowColor': '#1773c3',
                                         'shadowBlur': 20,
                                         'opacity': 0.8
                                         }
                              })
# 标签格式设置
bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=10,
                                              font_weight='bold',
                                              formatter='{b}:{c}'))
# 全局配置项
bar.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False, max_=5e6),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    title_opts=opts.TitleOpts(title="TOP 10", pos_top='55%', pos_left='5%',
                                              title_textstyle_opts=opts.TextStyleOpts(font_size=12)),
                    visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      max_=5e5,
                                                      is_piecewise=False,
                                                      dimension=0,
                                                      range_color=['rgba(219,112,147,0.4)', 'rgba(238,25,27,1)']))
# 转换xy轴
bar.reversal_axis()

"""
map图：
在整个grid中，map是被用来当作底图使用的
地图的颜色都在map中来配置
"""
mp = Map()
mp.add(
    "累计确诊人数",
    data_pair,
    "美国",
    is_map_symbol_show=False,
    is_roam=False)
# 关闭map的标签显示 & 地图风格设置
mp.set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                   itemstyle_opts={'normal': {
                       'areaColor': '#091632',
                       'borderColor': '#1773c3',
                       'shadowColor': '#1773c3',
                       'shadowBlur': 20,
                       'opacity': 0.6
                   }
})

mp.set_global_opts(
    title_opts=opts.TitleOpts(title="美国疫情地图 「确诊病例」", 
                              subtitle='数据更新日期：{}'.format(df_state.date.max()),
                              pos_top='2%', pos_left='center',
                              title_textstyle_opts=opts.TextStyleOpts(font_size=18)),
    legend_opts=opts.LegendOpts(is_show=False),
)

"""
GEO图：
geo在整个grid中是用于展示几个主要国家的数据
地图的样式都是由map控制
"""
data_pair = [[x, y] for x, y in data_pair if x in state_list]
geo = Geo()

# 需要先将几个国家的经纬度信息加入到geo中
for k, v in loc.items():
    geo.add_coordinate(k, v[0], v[1])
# 这里将geo的地图透明度配置为0
geo.add_schema(
    maptype="美国",
    is_roam=False,
    itemstyle_opts={
        'normal': {
            'opacity': 0}})

geo.add("", data_pair, symbol_size=1)
# 显示标签配置
geo.set_series_opts(
    label_opts=opts.LabelOpts(
        is_show=True,
        position='right',
        color='white',
        font_size=12,
        font_weight='bold',
        formatter=JsCode(fmt_js)),
)

# 将三个图组合起来，配置好各自的位置
grid = (
    Grid(init_opts=opts.InitOpts(theme='chalk', width='1000px', height='600px'))
    .add(bar, grid_opts=opts.GridOpts(pos_top="60%", pos_right='70%', pos_left='5%'))
    .add(mp, grid_opts=opts.GridOpts(pos_top="12%"))
    .add(geo, grid_opts=opts.GridOpts(pos_bottom="12%"))
)

grid.render_notebook()

#### 死亡

In [5]:
# 加载美国地图
register_url("https://echarts-maps.github.io/echarts-countries-js/")

state_list = ['California', 'Texas', 'New York', 'Florida', 'Arizona', 'Illinois']
loc = {
    'California': [-125.24302, 40.059232],
    'New York': [-77.875175, 42.850458],
    'Florida': [-82.269594, 27.785481],
    'Texas': [-101.102854, 31.916611],
    'Arizona': [-113.24302, 35.059232],
    'Illinois': [-90.5213, 39.8425]
}


data_new = df_state[df_state.date == df_state.date.max()]

data_pair = []
for _, row in data_new.iterrows():
    data_pair.append([row.state, row.deaths])

# 用于geo和bar中的标签显示
fmt_js = """function (params) {return params.name+':'+Number(params.value[2]);}"""

"""
Bar:
展示每天top10国家的数据
放在grid的左下角
"""
t = sorted(data_pair, key=lambda x: x[1], reverse=True)[:10][::-1]

bar = Bar()
bar.add_xaxis([x for x, y in t])
bar.add_yaxis("", [y for x, y in t],
              itemstyle_opts={'normal': {'borderColor': '#1773c3',
                                         'shadowColor': '#1773c3',
                                         'shadowBlur': 20,
                                         'opacity': 0.8
                                         }
                              })
# 标签格式设置
bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=10,
                                              font_weight='bold',
                                              formatter='{b}:{c}'))
# 全局配置项
bar.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False, max_=6e4),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    title_opts=opts.TitleOpts(title="TOP 10", pos_top='55%', pos_left='5%',
                                              title_textstyle_opts=opts.TextStyleOpts(font_size=12)),
                    visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      max_=1e4,
                                                      is_piecewise=False,
                                                      dimension=0,
                                                      range_color=['rgba(219,112,147,0.4)', 'rgba(238,25,27,1)']))
# 转换xy轴
bar.reversal_axis()

"""
map图：
在整个grid中，map是被用来当作底图使用的
地图的颜色都在map中来配置
"""
mp = Map()
mp.add(
    "累计确诊人数",
    data_pair,
    "美国",
    is_map_symbol_show=False,
    is_roam=False)
# 关闭map的标签显示 & 地图风格设置
mp.set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                   itemstyle_opts={'normal': {
                       'areaColor': '#091632',
                       'borderColor': '#1773c3',
                       'shadowColor': '#1773c3',
                       'shadowBlur': 20,
                       'opacity': 0.6
                   }
})

mp.set_global_opts(
    title_opts=opts.TitleOpts(title="美国疫情地图 「死亡病例」", 
                              subtitle='数据更新日期：{}'.format(df_state.date.max()),
                              pos_top='2%', pos_left='center',
                              title_textstyle_opts=opts.TextStyleOpts(font_size=18)),
    legend_opts=opts.LegendOpts(is_show=False),
)

"""
GEO图：
geo在整个grid中是用于展示几个主要国家的数据
地图的样式都是由map控制
"""
data_pair = [[x, y] for x, y in data_pair if x in state_list]
geo = Geo()

# 需要先将几个国家的经纬度信息加入到geo中
for k, v in loc.items():
    geo.add_coordinate(k, v[0], v[1])
# 这里将geo的地图透明度配置为0
geo.add_schema(
    maptype="美国",
    is_roam=False,
    itemstyle_opts={
        'normal': {
            'opacity': 0}})

geo.add("", data_pair, symbol_size=1)
# 显示标签配置
geo.set_series_opts(
    label_opts=opts.LabelOpts(
        is_show=True,
        position='right',
        color='white',
        font_size=12,
        font_weight='bold',
        formatter=JsCode(fmt_js)),
)

# 将三个图组合起来，配置好各自的位置
grid = (
    Grid(init_opts=opts.InitOpts(theme='chalk', width='1000px', height='600px'))
    .add(bar, grid_opts=opts.GridOpts(pos_top="60%", pos_right='70%', pos_left='5%'))
    .add(mp, grid_opts=opts.GridOpts(pos_top="12%"))
    .add(geo, grid_opts=opts.GridOpts(pos_bottom="12%"))
)

grid.render_notebook()

### 疫情趋势

#### 累计情况

In [6]:
background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: 'rgba(25,25,25, 0.8)'}, {offset: 1, color: 'rgba(139,0,0,0.5)'}], false)"
)

line_style = {
    'normal': {
        'width': 4,
        'shadowColor': 'rgba(155, 18, 184, .5)',
        'shadowBlur': 10,
        'shadowOffsetY': 20,
        'shadowOffsetX': 20,
        'curve':0.5
    }
}
     
line = (
    Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js), width='1000px', height='600px'))
    .add_xaxis(df_total.date.tolist())
    .add_yaxis("累计确诊", 
        df_total.cases.tolist(), 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )
    .add_yaxis("累计死亡", 
        df_total.deaths.tolist(), 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )
    )
line.set_series_opts(linestyle_opts=line_style,
                     label_opts=opts.LabelOpts(is_show=False, position="top", color="white"),
                     tooltip_opts=opts.TooltipOpts(is_show=True))
line.set_global_opts(title_opts=opts.TitleOpts(title="美国疫情趋势",
                                               subtitle='数据更新日期：{}'.format(df_total.date.max()),
                                                pos_left="center",
                                                title_textstyle_opts=opts.TextStyleOpts(color="white", font_size=20),),
                     legend_opts=opts.LegendOpts(is_show=True, pos_top='5%', pos_left="80%", 
                                                 selected_mode='single',legend_icon='circle',
                                                 textstyle_opts=opts.TextStyleOpts(color="white", font_size=10)),
                     xaxis_opts=opts.AxisOpts(boundary_gap=False,
                                                axislabel_opts=opts.LabelOpts(margin=30, color="#ffffff63",
                                                                              ),
                                                axisline_opts=opts.AxisLineOpts(is_show=False),
                                                axistick_opts=opts.AxisTickOpts(
                                                    is_show=True,
                                                    length=25,
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                                ),
                                                splitline_opts=opts.SplitLineOpts(
                                                    is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                                ),
                                            ),
                    yaxis_opts=opts.AxisOpts(
                                            type_="value",
                                            position="right",
                                            axislabel_opts=opts.LabelOpts(margin=20, color="#ffffff63",
                                                                          formatter=JsCode("""function (value) 
                                                                               {return value / 10000 +'万';}""")),
                                            axisline_opts=opts.AxisLineOpts(
                                                linestyle_opts=opts.LineStyleOpts(width=2, color="#fff")
                                            ),
                                            axistick_opts=opts.AxisTickOpts(
                                                is_show=True,
                                                length=15,
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                            ),
                                            splitline_opts=opts.SplitLineOpts(
                                                is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                            ),
                                        ),)      
line.render_notebook()

#### 每日新增

In [7]:
data_x, data_y_1, data_y_2 = [], [], []
for i in range(1, df_total.shape[0]):
    data_x.append(df_total.iloc[i, 0])
    data_y_1.append(int(df_total.iloc[i, 1] - df_total.iloc[i-1, 1]))
    data_y_2.append(int(df_total.iloc[i, 2] - df_total.iloc[i-1, 2]))
    
    
background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: 'rgba(25,25,25, 0.8)'}, {offset: 1, color: 'rgba(139,0,0,0.5)'}], false)"
)

line_style = {
    'normal': {
        'width': 4,
        'shadowColor': 'rgba(155, 18, 184, .5)',
        'shadowBlur': 10,
        'shadowOffsetY': 20,
        'shadowOffsetX': 20,
        'curve':0.5
    }
}
     
line = (
    Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js), width='1000px', height='600px'))
    .add_xaxis(data_x)
    .add_yaxis("新增确诊", 
        data_y_1, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )
    .add_yaxis("新增死亡", 
        data_y_2, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )
    )
line.set_series_opts(linestyle_opts=line_style,
                     label_opts=opts.LabelOpts(is_show=False, position="top", color="white"),
                     tooltip_opts=opts.TooltipOpts(is_show=True))
line.set_global_opts(title_opts=opts.TitleOpts(title="美国疫情新增趋势",
                                               subtitle='数据更新日期：{}'.format(df_total.date.max()),
                                               pos_left="center",
                                               title_textstyle_opts=opts.TextStyleOpts(color="white", font_size=20),),
                     legend_opts=opts.LegendOpts(is_show=True, pos_top='5%', pos_left="80%", 
                                                 selected_mode='single',legend_icon='circle',
                                                 textstyle_opts=opts.TextStyleOpts(color="white", font_size=10)),
                     xaxis_opts=opts.AxisOpts(boundary_gap=False,
                                                axislabel_opts=opts.LabelOpts(margin=30, color="#ffffff63"),
                                                axisline_opts=opts.AxisLineOpts(is_show=False),
                                                axistick_opts=opts.AxisTickOpts(
                                                    is_show=True,
                                                    length=25,
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                                ),
                                                splitline_opts=opts.SplitLineOpts(
                                                    is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                                ),
                                            ),
                    yaxis_opts=opts.AxisOpts(
                                            type_="value",
                                            position="right",
                                            axislabel_opts=opts.LabelOpts(margin=20, color="#ffffff63"),
                                            axisline_opts=opts.AxisLineOpts(
                                                linestyle_opts=opts.LineStyleOpts(width=2, color="#fff")
                                            ),
                                            axistick_opts=opts.AxisTickOpts(
                                                is_show=True,
                                                length=15,
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                            ),
                                            splitline_opts=opts.SplitLineOpts(
                                                is_show=True, linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                            ),
                                        )
                                        )      
line.render_notebook()

#### 蔓延趋势

In [8]:
# 加载美国地图
register_url("https://echarts-maps.github.io/echarts-countries-js/")

state_list = ['California', 'Texas', 'New York', 'Florida', 'Arizona', 'Illinois']
loc = {
    'California': [-125.24302, 40.059232],
    'New York': [-77.875175, 42.850458],
    'Florida': [-82.269594, 27.785481],
    'Texas': [-101.102854, 31.916611],
    'Arizona': [-113.24302, 35.059232],
    'Illinois': [-90.5213, 39.8425]
}

# 风格以及宽高配置
tl = Timeline(init_opts=opts.InitOpts(theme='chalk', width='1000px', height='600px'))

# 时间轴配置
tl.add_schema(
    is_auto_play=True,  # 自动播放
    is_timeline_show=False,   # 关闭timeline组件
    is_loop_play=True,  # 是否循环播放
    play_interval=200   # 播放速度，200ms
    )
    
for day in df_state.date.unique().tolist()[::5]:
    data_new = df_state[df_state.date == day]
    
    data_pair = []
    for _, row in data_new.iterrows():
        data_pair.append([row.state, row.cases])
    
    # 用于geo和bar中的标签显示
    fmt_js = """function (params) {return params.name+':'+Number(params.value[2]);}"""
    
    """
    Bar:
    展示每天top10国家的数据
    放在grid的左下角
    """
    t = sorted(data_pair, key=lambda x: x[1], reverse=True)[:10][::-1]
    
    bar = Bar()
    bar.add_xaxis([x for x, y in t])
    bar.add_yaxis("", [y for x, y in t],
                  itemstyle_opts={'normal': {'borderColor': '#1773c3',
                                             'shadowColor': '#1773c3',
                                             'shadowBlur': 20,
                                             'opacity': 0.8
                                             }
                                  })
    # 标签格式设置
    bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                                  font_size=10,
                                                  font_weight='bold',
                                                  formatter='{b}:{c}'))
    # 全局配置项
    bar.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False, max_=5e6),
                        yaxis_opts=opts.AxisOpts(is_show=False),
                        title_opts=opts.TitleOpts(title="TOP 10 {}".format(day), pos_top='55%', pos_left='5%',
                                                  title_textstyle_opts=opts.TextStyleOpts(font_size=12)),
                        visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                          max_=5e5,
                                                          is_piecewise=False,
                                                          dimension=0,
                                                          range_color=['rgba(219,112,147,0.4)', 'rgba(238,25,27,1)']))
    # 转换xy轴
    bar.reversal_axis()
    
    """
    map图：
    在整个grid中，map是被用来当作底图使用的
    地图的颜色都在map中来配置
    """
    mp = Map()
    mp.add(
        "累计确诊人数",
        data_pair,
        "美国",
        is_map_symbol_show=False,
        is_roam=False)
    # 关闭map的标签显示 & 地图风格设置
    mp.set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                       itemstyle_opts={'normal': {
                           'areaColor': '#091632',
                           'borderColor': '#1773c3',
                           'shadowColor': '#1773c3',
                           'shadowBlur': 20,
                           'opacity': 0.6
                       }
    })
    
    mp.set_global_opts(
        title_opts=opts.TitleOpts(title="美国疫情蔓延趋势", 
                                  subtitle='数据更新日期：{}'.format(df_state.date.max()),
                                  pos_top='2%', pos_left='center',
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=18)),
        legend_opts=opts.LegendOpts(is_show=False),
    )
    
    """
    GEO图：
    geo在整个grid中是用于展示几个主要国家的数据
    地图的样式都是由map控制
    """
    data_pair = [[x, y] for x, y in data_pair if x in state_list]
    geo = Geo()
    
    # 需要先将几个国家的经纬度信息加入到geo中
    for k, v in loc.items():
        geo.add_coordinate(k, v[0], v[1])
    # 这里将geo的地图透明度配置为0
    geo.add_schema(
        maptype="美国",
        is_roam=False,
        itemstyle_opts={
            'normal': {
                'opacity': 0}})
    
    geo.add("", data_pair, symbol_size=1)
    # 显示标签配置
    geo.set_series_opts(
        label_opts=opts.LabelOpts(
            is_show=True,
            position='right',
            color='white',
            font_size=12,
            font_weight='bold',
            formatter=JsCode(fmt_js)),
    )
    
    # 将三个图组合起来，配置好各自的位置
    grid = (
        Grid(init_opts=opts.InitOpts(theme='chalk', width='1000px', height='600px'))
        .add(bar, grid_opts=opts.GridOpts(pos_top="60%", pos_right='70%', pos_left='5%'))
        .add(mp, grid_opts=opts.GridOpts(pos_top="12%"))
        .add(geo, grid_opts=opts.GridOpts(pos_bottom="12%"))
    )
    
    grid.render_notebook()
    
    tl.add(grid, day)

tl.render_notebook()

### 城市情况

#### 概览

In [9]:
data_new = df_city[(df_city.date == df_city.date.max()) & (df_city.cases > 30000)].copy()
data_new['death_rate'] = data_new['deaths'] / data_new['cases']

data_x, data_y = [], []
for idx, row in data_new.iterrows():
    data_y.append([row['death_rate'], row['deaths'], row['date'], row['county']])
    data_x.append(row['cases'])


tool_js = """
function (obj) {
            var value = obj.value;
            var schema = [{name: 'cases', index: 0, text: '确诊病例'},
                          {name: 'deaths', index: 1, text: '死亡病例'},
                          {name: 'death_rate', index: 2, text: '死亡率'}];
            return '<div style="border-bottom: 1px solid rgba(255,255,255,.3); font-size: 18px;padding-bottom: 7px;margin-bottom: 7px">'
                + value[4] + '<br>'
                + value[3]
                + '</div>'
                + schema[0].text + '：' + value[0] + '<br>'
                + schema[1].text + '：' + value[2] + '<br>'
                + schema[2].text + '：' + value[1].toFixed(4)*100+'%' + '<br>';
        }
"""

color=JsCode("""new echarts.graphic.RadialGradient(0.4, 0.3, 1, [{
                                        offset: 0,
                                        color: 'rgb(251, 118, 123)'
                                    }, {
                                        offset: 1,
                                        color: 'rgb(204, 46, 72)'
                                    }])""")

itemStyle = {
    'opacity': 0.8,
    'shadowBlur': 10,
    'shadowOffsetX': 10,
    'shadowOffsetY': 10,
    'shadowColor': 'rgba(0, 0, 0, 0.5)',
    'color': color
}



visualMap = [
        {
            'left': 'right',
            'top': '35%',
            'dimension': 2,
            'min': 0,
            'max': 5000,
            'itemWidth': 30,
            'itemHeight': 120,
            'calculable': True,
            'precision': 0.1,
            'text': ['圆形大小：死亡病例'],
            'textGap': 30,
            'textStyle': {
                'color': '#fff'
            },
            'inRange': {
                'symbolSize': [5, 50]
            },
            'controller': {
                'inRange': {
                    'color': ['rgb(251, 118, 123)']
                },
                'outOfRange': {
                    'color': ['#444']
                }
            }
        }
    ]

scatter = (Scatter(init_opts=opts.InitOpts(theme='chalk' ,width='1000px', height='600px'))
            .add_xaxis(data_x)
            .add_yaxis("", data_y,
                       label_opts=opts.LabelOpts(is_show=False),
                       itemstyle_opts=itemStyle
                       )
            .set_global_opts(yaxis_opts=opts.AxisOpts(name='死亡率', type_="value", is_scale=True, 
                                                      name_textstyle_opts={'color': '#fff', 'fontSize': 16},
                                                      axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#eee')),
                                                      axislabel_opts=opts.LabelOpts(margin=20, 
                                                                          formatter=JsCode("""function (value) 
                                                                               {return value * 100 +'%';}""")),),
                             xaxis_opts=opts.AxisOpts(name='确诊数', type_="value", min_=5e3, 
                                                      name_textstyle_opts={'color': '#fff', 'fontSize': 16},
                                                      axisline_opts=opts.AxisLineOpts(linestyle_opts=opts.LineStyleOpts(color='#eee'))),
                             tooltip_opts=opts.TooltipOpts(is_show=True, background_color='#222', border_color='#777', border_width=1, formatter=JsCode(tool_js)),
                             legend_opts=opts.LegendOpts(is_show=True, pos_right=10,
                                                         textstyle_opts=opts.TextStyleOpts(color='#fff',font_size=16)),
                             visualmap_opts=visualMap,
                             title_opts=opts.TitleOpts(title="美国城市疫情情况", 
                                  subtitle='数据更新日期：{}'.format(df_city.date.max()),
                                  pos_top='1%', pos_left='center',
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=18)),
                             )
            )

            

scatter.render_notebook()

#### 确诊TOP

In [10]:
data_new = data_new.sort_values(by='cases', ascending=False)

bar = Bar(init_opts=opts.InitOpts(theme='chalk' ,width='1000px', height='800px'))
bar.add_xaxis(data_new['county'][:20].tolist()[::-1])
bar.add_yaxis("", data_new['cases'][:20].tolist()[::-1],
              itemstyle_opts={'normal': {'borderColor': '#fff',
                                         'borderWidth': 2,
                                         'shadowBlur': 1,
                                         'opacity': 0.8
                                         }
                              })
# 标签格式设置
bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=12,
                                              font_weight='bold',
                                              formatter='{b}:{c}'))
# 全局配置项
bar.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    title_opts=opts.TitleOpts(title="确诊TOP 20", 
                                  subtitle='数据更新日期：{}'.format(df_city.date.max()),
                                  pos_top='80%', pos_left='70%',
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=28)),
                    visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      max_=1e6,
                                                      min_=200,
                                                      is_piecewise=False,
                                                      dimension=0,
                                                      range_color=['rgba(238,25,27,1)', 'rgba(219,112,147,0.4)']),
                    graphic_opts=[
                            opts.GraphicGroup(
                                graphic_item=opts.GraphicItem(left="5%", top="5%"),
                                children=[
                                    opts.GraphicRect(
                                        graphic_item=opts.GraphicItem(
                                            z=0, left="center", top="middle"
                                        ),
                                        graphic_shape_opts=opts.GraphicShapeOpts(width=900, height=720),
                                        graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                            fill="rgba(0,0,0,0)",
                                            stroke="#fff",
                                            line_width=2,
                                            shadow_blur=20,
                                            shadow_offset_x=5,
                                            shadow_offset_y=5,
                                            shadow_color="#4169E1",
                                        ),
                                    ),
                                ],
                            )
                        ]
                    )
# 转换xy轴
bar.reversal_axis()
bar.render_notebook()

#### 新增TOP

In [11]:
lastest_day, past_day = df_city.date.unique()[-1], df_city.date.unique()[-2]

df_1 = df_city[df_city.date == lastest_day].copy()
df_2 = df_city[df_city.date == past_day].copy()

df_add = pd.merge(df_1, df_2, how='inner', on=['county', 'state'])
df_add = df_add[df_add['county'] != 'Unknown']
df_add['add_cases'] = df_add['cases_x'] - df_add['cases_y']
df_add = df_add.sort_values(by='add_cases', ascending=False)

bar = Bar(init_opts=opts.InitOpts(theme='chalk' ,width='1000px', height='800px'))
bar.add_xaxis(df_add['county'][:20].tolist()[::-1])
bar.add_yaxis("", df_add['add_cases'][:20].tolist()[::-1],
              itemstyle_opts={'normal': {'borderColor': '#fff',
                                         'borderWidth': 2,
                                         'shadowBlur': 1,
                                         'opacity': 0.8
                                         }
                              })
# 标签格式设置
bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=12,
                                              font_weight='bold',
                                              formatter='{b}:{c}'))
# 全局配置项
bar.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    title_opts=opts.TitleOpts(title="新增TOP 20", 
                                  subtitle='数据更新日期：{}'.format(df_city.date.max()),
                                  pos_top='80%', pos_left='70%',
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=28)),
                    visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      max_=10000,
                                                      min_=200,
                                                      is_piecewise=False,
                                                      dimension=0,
                                                      range_color=['rgba(238,25,27,1)', 'rgba(219,112,147,0.4)']),
                    graphic_opts=[
                            opts.GraphicGroup(
                                graphic_item=opts.GraphicItem(left="5%", top="5%"),
                                children=[
                                    opts.GraphicRect(
                                        graphic_item=opts.GraphicItem(
                                            z=0, left="center", top="middle"
                                        ),
                                        graphic_shape_opts=opts.GraphicShapeOpts(width=900, height=720),
                                        graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                            fill="rgba(0,0,0,0)",
                                            stroke="#fff",
                                            line_width=2,
                                            shadow_blur=20,
                                            shadow_offset_x=5,
                                            shadow_offset_y=5,
                                            shadow_color="#4169E1",
                                        ),
                                    ),
                                ],
                            )
                        ]
                    )
# 转换xy轴
bar.reversal_axis()
bar.render_notebook()

***❤️ 🧡 💜 欢迎点赞，Fork～～***